# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Load the PRICE_DATA environment variable
price_data = os.getenv("PRICE_DATA")
print(price_data)

# Find all parquet files in that directory
parquet_files = glob(os.path.join(price_data, "*.parquet"))

print(f"Found {len(parquet_files)} parquet files.")



../../05_src/data/temp/parquet/
Found 60 parquet files.


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
ddf0 = dd.read_parquet(parquet_files[0])
print(ddf0.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker'],
      dtype='object')


In [13]:
# Write your code below.

import dask.dataframe as dd

dd_list = []

for fp in parquet_files:
    # Read one ticker's parquet file
    ddf = dd.read_parquet(fp)

    # Sort chronologically (required for correct lag/shift)
    ddf = ddf.map_partitions(lambda pdf: pdf.sort_values("Date"))

    # Add lag features (previous row within this ticker file)
    ddf["Close_lag_1"] = ddf["Close"].shift(1)
    ddf["Adj Close_lag_1"] = ddf["Adj Close"].shift(1)

    # Add returns based on Close
    ddf["returns"] = (ddf["Close"] / ddf["Close_lag_1"]) - 1

    # Add daily High–Low range
    ddf["hi_lo_range"] = ddf["High"] - ddf["Low"]

    dd_list.append(ddf)

# Combine all tickers into a single Dask dataframe
dd_feat = dd.concat(dd_list, interleave_partitions=True)

dd_feat


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Close_lag_1,Adj Close_lag_1,returns,hi_lo_range
npartitions=60,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,string,string,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

# Convert the Dask dataframe to a pandas dataframe
df_feat = dd_feat.compute()

# Add a 10-day moving average of returns
df_feat["returns_ma_10"] = df_feat["returns"].rolling(10).mean()

df_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Close_lag_1,Adj Close_lag_1,returns,hi_lo_range,returns_ma_10
0,1973-02-22,4.25,4.250,4.25,4.25,0.507389,800.0,TNC.csv,TNC,NaN,NaN,NaN,0.000,NaN
1,1973-02-23,4.25,4.250,4.25,4.25,0.507389,800.0,TNC.csv,TNC,4.25,0.507389,0.000000,0.000,NaN
2,1973-02-26,4.25,4.250,4.25,4.25,0.507389,800.0,TNC.csv,TNC,4.25,0.507389,0.000000,0.000,NaN
3,1973-02-27,4.25,4.250,4.25,4.25,0.507389,0.0,TNC.csv,TNC,4.25,0.507389,0.000000,0.000,NaN
4,1973-02-28,4.25,4.250,4.25,4.25,0.507389,0.0,TNC.csv,TNC,4.25,0.507389,0.000000,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36773,2020-03-30,0.62,0.680,0.60,0.67,0.670000,980200.0,REI.csv,REI,12.30,12.300000,-0.945528,0.080,5.835276
38066,2020-03-31,11.78,12.400,10.50,10.96,10.960000,926900.0,BLPH.csv,BLPH,0.67,0.670000,15.358209,1.900,7.465733
36774,2020-03-31,0.67,0.700,0.65,0.66,0.660000,508300.0,REI.csv,REI,10.96,10.960000,-0.939781,0.050,6.056371
36775,2020-04-01,0.65,0.650,0.53,0.54,0.540000,1239600.0,REI.csv,REI,0.66,0.660000,-0.181818,0.120,6.130363


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

not necessary while Dask supports rolling operations and moving averages itself.

+ Would it have been better to do it in Dask? Why?

yes, it is more efficient. Dask is better for large datsets as it performs the operation computation distribution and memory efficient while pandas load the entire dataset to memory.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.